In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from IPython.display import Image

from config import (SPECS_FLOOD_YAMLS, CONSOLIDATED_RESULTS_DIR, FIG_RESULTS_DIR,
                    SP_RESULTS_DIR, RO_RESULTS_DIR,
                    SPSOL_ROMOD_RESULTS_DIR, ROSOL_SPMOD_RESULTS_DIR)
from drawutil import utcolors, mpl_config
from model import Solution

In [ ]:
# global setting
SCALE = 1 # scale = 1.0 for write-up dimensions
mpl_config(SCALE)

In [ ]:
casestudies = [
    'imelda',
    'harvey',
]

f_mins = {
    ('stochastic', 'imelda'): 0,
    ('robust', 'imelda'): 0,
    ('stochastic', 'harvey'): 0,
    ('robust', 'harvey'): 0,
}

f_maxs = {
    ('stochastic', 'imelda'): 20,
    ('robust', 'imelda'): 9,
    ('stochastic', 'harvey'): 193,
    ('robust', 'harvey'): 62,
}

r_hat = 3

xlim = {
    ('stochastic', 'imelda'): (0 - 20 * 0.05, 20 + 20 * 0.05),
    ('robust', 'imelda'): (0 - 10 * 0.05, 10 + 10 * 0.05),
    ('stochastic', 'harvey'): (0 - 200 * 0.05, 200 + 200 * 0.05),
    ('robust', 'harvey'): (0 - 60 * 0.05, 60 + 60 * 0.05),
}

In [ ]:
# plot sizing overhead
plot_num_w = 2
plot_num_h = 2
lo_pad_w, md_pad_w, hi_pad_w, plot_dim_w = 0.625 * SCALE, 0.500 * SCALE, 0.250 * SCALE, 2.500 * SCALE
lo_pad_h, md_pad_h, hi_pad_h, plot_dim_h = 0.500 * SCALE, 0.125 * SCALE, 0.250 * SCALE, 1.750 * SCALE
fig_w = lo_pad_w + hi_pad_w + md_pad_w * (plot_num_w - 1) + plot_dim_w * plot_num_w
fig_h = lo_pad_h + hi_pad_h + md_pad_h * (plot_num_h - 1) + plot_dim_h * plot_num_h
plot_num_w = 2
plot_num_h = 2
lo_pad_w, md_pad_w, hi_pad_w, plot_dim_w = 0.625 * SCALE, 0.125 * SCALE, 0.250 * SCALE, 2.750 * SCALE
lo_pad_h, md_pad_h, hi_pad_h, plot_dim_h = 0.500 * SCALE, 0.125 * SCALE, 0.250 * SCALE, 1.750 * SCALE
fig_w = lo_pad_w + hi_pad_w + md_pad_w * (plot_num_w - 1) + plot_dim_w * plot_num_w
fig_h = lo_pad_h + hi_pad_h + md_pad_h * (plot_num_h - 1) + plot_dim_h * plot_num_h
fig, axes = plt.subplots(plot_num_h, plot_num_w,
                         figsize=(fig_w, fig_h),
                         sharex='col',
                         sharey='row')
plt.subplots_adjust(left=lo_pad_w / fig_w,
                    bottom=lo_pad_h / fig_h,
                    right=(fig_w - hi_pad_w) / fig_w,
                    top=(fig_h - hi_pad_h) / fig_h,
                    wspace=md_pad_w / plot_dim_w,
                    hspace=md_pad_h / plot_dim_h)


for col, casestudy in enumerate(casestudies):

    pftype = 'dc'
    storm = casestudy.split('-')[0].title()
    with open(SPECS_FLOOD_YAMLS[casestudy,]) as fh:
        specs = yaml.load(fh, Loader=yaml.Loader)
        for key, val in specs['r_hat'].items():
            specs['r_hat'][key] = min(val, r_hat)
        for key in list(specs['xi']):
            (k, r, omega) = key
            if r > r_hat:
                specs['xi'].pop(key)
        for k in specs['R']:
            specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]
        c = pd.Series(specs['c'])
        c = c.loc[c.index.get_level_values(1) <= r_hat]
    x_path = os.path.join(CONSOLIDATED_RESULTS_DIR, f'x-{casestudy}-r{r_hat}.csv')
    df_x = pd.read_csv(x_path, index_col=[0, 1], header=[0, 1, 2, 3])
    df_x.index.names = ['k', 'r']
    df_x.columns.names = ['model', 'casestudy', 'pftype', 'f']

    f_min = 1
    f_max_sp = f_maxs['stochastic', casestudy]
    f_max_ro = f_maxs['robust', casestudy]
    f_max = max(f_max_sp, f_max_ro)
    f_inc = 1

    xs = []
    ys = []
    for f in np.arange(f_min, f_max + f_inc, f_inc):
        sr_sp = df_x[('SP', casestudy, pftype, str(min(f, f_max_sp)))].dropna().round().astype(int)
        sr_ro = df_x[('RO', casestudy, pftype, str(min(f, f_max_ro)))].dropna().round().astype(int)
        xs.append(f)
        ys.append(sr_sp.multiply(sr_ro).multiply(c).sum() / max(sr_sp.multiply(c).sum(), sr_ro.multiply(c).sum()))
    axes[0, col].plot(xs, ys, '.', color='k')

    for f in np.arange(f_min, f_max + f_inc, f_inc):
        sol_filename = os.path.join(SP_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
        if os.path.exists(sol_filename):
            z_best = Solution.from_zip(sol_filename)['ObjVal']
        f_get = min(f, f_max_ro)
        sol_filename = os.path.join(ROSOL_SPMOD_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
        if os.path.exists(sol_filename):
            z_ro = Solution.from_zip(sol_filename)['ObjVal']
        label = r'$\boldsymbol{x}_\text{RO}^*$ in SP Model' if f == f_min else None
        axes[1, col].plot(f, (z_ro - z_best) / z_best, '.', color=utcolors[1], label=label)

    for f in np.arange(f_min, f_max + f_inc, f_inc):
        sol_filename = os.path.join(RO_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
        if os.path.exists(sol_filename):
            z_best = Solution.from_zip(sol_filename)['ObjVal']
        f_get = min(f, f_max_sp)
        sol_filename = sol_filename = os.path.join(SPSOL_ROMOD_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
        if os.path.exists(sol_filename):
            z_sp = Solution.from_zip(sol_filename)['ObjVal']
        label = r'$\boldsymbol{x}_\text{SP}^*$ in RO Model' if f == f_min else None
        axes[1, col].plot(f, (z_sp - z_best) / z_best, '.', color=utcolors[5], label=label)

    axes[0, col].set_xlim(xlim['stochastic', casestudy])
    axes[0, col].set_ylim(0 - 1 * 0.10, 1 + 1 * 0.10)
    axes[1, col].set_ylim(0 - 0.4 * 0.10, 0.4 + 0.4 * 0.10)
    axes[0, col].set_title(storm)
    axes[-1, col].set_xlabel('$f$')
    axes[1, col].legend(loc='upper left')


axes[0, 0].set_ylabel(r'$\text{RelSim}(\boldsymbol{x}_\text{SP}^*, \boldsymbol{x}_\text{RO}^*; \boldsymbol{c})$')
axes[1, 0].set_ylabel('Relative Optimality Gap')

image_jpg = os.path.join(FIG_RESULTS_DIR, f'ijoc-sp-ro-similarity.jpg')
plt.savefig(image_jpg, format='jpg', dpi=256)
plt.close()
display(Image(image_jpg))